<a href="https://colab.research.google.com/github/russell-ai/topic-covid/blob/dev/tweet_topic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [1]:
!pip install bertopic
#!pip install bertopic --no-build-isolation --no-binary :all: # in case of conflicting with numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# load data

In [ ]:
#Dont run dataset for before sentiment anaysis.
#import gdown
#url = "https://drive.google.com/file/d/1sjtke6Y_3wMlBQoYCHoANQu7Fj7l7rrn/view?usp=share_link"
#gdown.download(url=url,fuzzy=True)
#gdown.download_folder(url, quiet=True, use_cookies=False)

In [3]:
#@title Drive Downloader

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

download_with_pydrive = True #@param {type:"boolean"}  

class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()
        
    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_id, file_dst):
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id file_dst

downloader = Downloader(download_with_pydrive)

In [4]:
current_directory = os.getcwd()
save_path = os.path.join(os.path.dirname(current_directory), "data")
os.makedirs(save_path, exist_ok=True)
model_path = os.path.join(save_path, 'sent_tweet_data.csv')

In [5]:
downloader.download_file("1k8HXp4vUBBbVqKu3kBYnu-HLDQdAdzrf", file_dst=model_path)

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("/data/sent_tweet_data.csv")
data.head()

,Tweet_Id,Tweet,sentiment_analysis,sentiment,score
0,1.598104e+18,@elonmusk I thunk you're an irresponsible for ...,"('negative', 0.8987942934036255)",negative,0.898794
1,1.598104e+18,"Apparently ""I've had COVID for over a week and...","('negative', 0.5844972133636475)",negative,0.584497
2,1.598103e+18,@snakewasright @EllenOl00313689 @iTruthSearch ...,"('negative', 0.7049756646156311)",negative,0.704976
3,1.598103e+18,They're dropping the sequel to COVID baby wooo...,"('neutral', 0.5086733102798462)",neutral,0.508673
4,1.598101e+18,@1goodtern They drop dead one by one of post C...,"('negative', 0.47382649779319763)",negative,0.473826


# positive data topic analysis

In [160]:
# get the tweet text and sentiment columns whre sentiment is positive as list
positive_data = data[data["sentiment"] == "positive"]
positive_data.head()


,Tweet_Id,Tweet,sentiment_analysis,sentiment,score
8,1.598098e+18,"My friend spencer has covid, good thing he ate...","('positive', 0.48467978835105896)",positive,0.484680
16,1.598092e+18,"""I had my baby shower here. Very big space, th...","('positive', 0.9333032369613647)",positive,0.933303
23,1.598088e+18,"Nat de covid 😭✨ Se cuide, baby https://t.co/yE...","('positive', 0.7633681893348694)",positive,0.763368
36,1.598079e+18,Get Well Soon my baby 🥹🍼 I know you're strong ...,"('positive', 0.947133481502533)",positive,0.947133
75,1.598065e+18,There’s a chosen family baby I know who’s just...,"('positive', 0.5353631377220154)",positive,0.535363


In [161]:
positive_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45263 entries, 8 to 446181
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Tweet_Id            45263 non-null  float64
 1   Tweet               45263 non-null  object 
 2   sentiment_analysis  45263 non-null  object 
 3   sentiment           45263 non-null  object 
 4   score               45263 non-null  float64
dtypes: float64(2), object(3)
memory usage: 2.1+ MB


## Some Tweet Cleaning

In [4]:
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
URL_REGEX = re.compile('http(s)?:\/\/t.co\/\w+')
MENTION_REGEX = re.compile('@\w+')

def clean_tweet(tweet):
    # remove mentions, the pound sign, and replace urls with URL token
    tweet = re.sub(URL_REGEX, '', tweet)
    tweet = re.sub(MENTION_REGEX, '', tweet)
    tweet = tweet.replace('#', '')
    tweet = re.sub(r'\d+', '', tweet)
    return tweet.strip()

def remove_stopwords(text):
    removed = []
    stop_words = list(stopwords.words("english"))
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)

def less_than_3(text):
    text = re.sub(r'\b\w{1,2}\b', '', text)
    return text

def lemmatizing(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        lemma_word = lemmatizer.lemmatize(tokens[i])
        tokens[i] = lemma_word
    return " ".join(tokens)


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def custom_clean(text):
    text = re.sub(r' amp ', ' ', text)
    text = re.sub(r' ampersand ' , ' ', text)
    text = re.sub(r' the ' , ' ', text)
    return text


def remove_emojis(text):
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    return wierd_pattern.sub(r'', text)

In [6]:
def tweet_cleaner(tweet):
    tweet = clean_tweet(tweet)
    tweet = remove_emojis(tweet)
    tweet = remove_stopwords(tweet)
    tweet = lemmatizing(tweet)
    tweet = less_than_3(tweet)
    tweet = remove_punctuation(tweet)
    tweet = custom_clean(tweet)
    return tweet

In [167]:
positive_data["cleaned_tweet"] = positive_data["Tweet"].apply(lambda s: tweet_cleaner(s))

In [168]:
positive_data.head()

,Tweet_Id,Tweet,sentiment_analysis,sentiment,score,cleaned_tweet
8,1.598098e+18,"My friend spencer has covid, good thing he ate...","('positive', 0.48467978835105896)",positive,0.484680,friend spencer covid good thing ate pound eg...
16,1.598092e+18,"""I had my baby shower here. Very big space, th...","('positive', 0.9333032369613647)",positive,0.933303,baby shower Very big space owner accommoda...
23,1.598088e+18,"Nat de covid 😭✨ Se cuide, baby https://t.co/yE...","('positive', 0.7633681893348694)",positive,0.763368,Nat covid cuide baby
36,1.598079e+18,Get Well Soon my baby 🥹🍼 I know you're strong ...,"('positive', 0.947133481502533)",positive,0.947133,Get Well Soon baby know strong COVID anyth...
75,1.598065e+18,There’s a chosen family baby I know who’s just...,"('positive', 0.5353631377220154)",positive,0.535363,There ’ chosen family baby know ’ cuddliest b...


In [169]:
positive_tweets = positive_data["cleaned_tweet"].tolist()

In [170]:
len(positive_tweets)

45263

## topic modelling

In [7]:
from bertopic import BERTopic

[Topic Model Spesifications:](https://www.sbert.net/docs/pretrained_models.html#other-models) 

In [172]:
topic_model = BERTopic(verbose=True, embedding_model="all-mpnet-base-v2",  calculate_probabilities=False)

In [173]:
topics, probs = topic_model.fit_transform(positive_tweets)

Batches:   0%|          | 0/1415 [00:00<?, ?it/s]

2022-12-26 20:51:17,409 - BERTopic - Transformed documents to Embeddings
2022-12-26 20:52:38,811 - BERTopic - Reduced dimensionality
2022-12-26 20:52:45,122 - BERTopic - Clustered reduced embeddings


In [174]:
freq = topic_model.get_topic_info()

In [175]:
freq.head()

,Topic,Count,Name
0,-1,25487,-1_covid_week_got_pregnant
1,0,696,0_biden_formula_trump_gas
2,1,543,1_study_research_review_article
3,2,423,2_help_bill_would_job
4,3,339,3_soon_well_rest_apo


In [177]:
topic_model.reduce_topics(positive_tweets, nr_topics=14)

2022-12-26 20:55:37,612 - BERTopic - Reduced number of topics from 463 to 15


In [178]:
import pprint
for i in range(3):
  pprint.pprint(topic_model.get_topic(i), indent=4)  # Select the most frequent topic

[   ('vaccine', 0.13060375755979678),
    ('protect', 0.08289921412562194),
    ('pregnant', 0.07508579863142656),
    ('safe', 0.060422845664195174),
    ('pregnancy', 0.058848096399361574),
    ('book', 0.05720146569649503),
    ('vaccination', 0.05640936536053837),
    ('covid', 0.05548396529896466),
    ('flu', 0.049594974074622745),
    ('way', 0.04865785371604313)]
[   ('pregnancy', 0.07906424305632108),
    ('covid', 0.052784194629744006),
    ('podcast', 0.04819691939104947),
    ('research', 0.04415941535675736),
    ('study', 0.04307084389607861),
    ('listen', 0.03458918567089253),
    ('great', 0.032994213604491805),
    ('health', 0.03277306592169822),
    ('maternal', 0.032487983406729974),
    ('researcher', 0.029911412801257985)]
[   ('help', 0.08295484159271078),
    ('would', 0.07241913650948226),
    ('due', 0.0672313274475425),
    ('baby', 0.04917719575379782),
    ('please', 0.04561481778027182),
    ('could', 0.04312939216344315),
    ('cash', 0.0429377140619615

## Topic Visualization

In [179]:
topic_model.visualize_topics()

In [180]:
topic_model.visualize_barchart(top_n_topics=14)

In [181]:
topic_model.visualize_barchart()

In [182]:
topic_model.visualize_heatmap(top_n_topics=14)

In [183]:
topic_model.visualize_hierarchy(top_n_topics=14)

In [184]:
topic_model.visualize_term_rank()

### save and load

In [176]:
topic_model.save("/content/drive/MyDrive/Colab Notebooks/PROJECTS/TOPIC-COVID/positive_model")

/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
#topic_model = BERTopic.load("/content/drive/MyDrive/Colab Notebooks/PROJECTS/TOPIC-COVID/positive_model")

# neutral topic analysis

In [15]:
# get the tweet text and sentiment columns whre sentiment is positive as list
neutral_data = data[data["sentiment"] == "neutral"]
neutral_data.head()


,Tweet_Id,Tweet,sentiment_analysis,sentiment,score
3,1.598103e+18,They're dropping the sequel to COVID baby wooo...,"('neutral', 0.5086733102798462)",neutral,0.508673
15,1.598093e+18,Baby 🦁\nCovid 👽,"('neutral', 0.7114819288253784)",neutral,0.711482
19,1.598090e+18,@DiedSuddenly_ In the 1950s I had polio. Spent...,"('neutral', 0.6331275701522827)",neutral,0.633128
27,1.598086e+18,Brianna has informed us that $0/$600 has been ...,"('neutral', 0.7449110150337219)",neutral,0.744911
29,1.598084e+18,@eternity_comics Is a baby gonna debate me? Is...,"('neutral', 0.5940708518028259)",neutral,0.594071


In [16]:
neutral_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160298 entries, 3 to 446215
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Tweet_Id            160298 non-null  float64
 1   Tweet               160298 non-null  object 
 2   sentiment_analysis  160298 non-null  object 
 3   sentiment           160298 non-null  object 
 4   score               160298 non-null  float64
dtypes: float64(2), object(3)
memory usage: 7.3+ MB


## Some Tweet Cleaning

In [17]:
neutral_data["cleaned_tweet"] = neutral_data["Tweet"].apply(lambda s: tweet_cleaner(s))

In [18]:
neutral_data.head()

,Tweet_Id,Tweet,sentiment_analysis,sentiment,score,cleaned_tweet
3,1.598103e+18,They're dropping the sequel to COVID baby wooo...,"('neutral', 0.5086733102798462)",neutral,0.508673,They dropping sequel COVID baby woooooooo
15,1.598093e+18,Baby 🦁\nCovid 👽,"('neutral', 0.7114819288253784)",neutral,0.711482,Baby Covid
19,1.598090e+18,@DiedSuddenly_ In the 1950s I had polio. Spent...,"('neutral', 0.6331275701522827)",neutral,0.633128,polio Spent summer bed first new Salk ...
27,1.598086e+18,Brianna has informed us that $0/$600 has been ...,"('neutral', 0.7449110150337219)",neutral,0.744911,Brianna informed raised today baby tested ...
29,1.598084e+18,@eternity_comics Is a baby gonna debate me? Is...,"('neutral', 0.5940708518028259)",neutral,0.594071,baby gon debate mock trial baby lawyer


In [19]:
neutral_data = neutral_data["cleaned_tweet"].tolist()

In [20]:
len(neutral_data)

160298

## topic modelling

In [9]:
topic_model = BERTopic(verbose=True, embedding_model="all-mpnet-base-v2",  calculate_probabilities=False)

In [194]:
topics, probs = topic_model.fit_transform(neutral_data)

Batches:   0%|          | 0/5010 [00:00<?, ?it/s]

2022-12-26 21:11:22,577 - BERTopic - Transformed documents to Embeddings
2022-12-26 21:21:20,702 - BERTopic - Reduced dimensionality
2022-12-26 21:21:49,907 - BERTopic - Clustered reduced embeddings


In [11]:
freq = topic_model.get_topic_info()

In [12]:
freq.head(20)

,Topic,Count,Name
0,-1,74550,-1_work_know_still_like
1,0,2491,0_biden_trump_abortion_pro
2,1,1154,1_petition_nhs_sign_partner
3,2,948,2_mask_wearing_wear_wore
4,3,945,3_lol_lmao_greysanatomy_everybody
5,4,671,4_kit_rapid_strip_cheap
6,5,668,5_sister_niece_nephew_fever
7,6,656,6_parliament_being_stop_from
8,7,648,7_black_bame_ethnic_minority
9,8,638,8_posting_pic_picture_posted


In [21]:
topic_model.reduce_topics(neutral_data, nr_topics=14)

2022-12-26 21:48:29,079 - BERTopic - Reduced number of topics from 2422 to 15


In [22]:
import pprint
for i in range(3):
  pprint.pprint(topic_model.get_topic(i), indent=4)  # Select the most frequent topic

[   ('pregnant', 0.07017797777216438),
    ('covid', 0.05893699750788093),
    ('woman', 0.05672259098354506),
    ('vaccine', 0.042527239162374275),
    ('pregnancy', 0.04165222863056176),
    ('get', 0.03362786964537694),
    ('vaccination', 0.031681166328945305),
    ('risk', 0.0309551498673272),
    ('walk', 0.027267606022700935),
    ('dying', 0.026299647851068593)]
[   ('trump', 0.05291414536830168),
    ('abortion', 0.035054910046076034),
    ('biden', 0.032493582209463356),
    ('covid', 0.03171677712511955),
    ('birth', 0.02915251107915654),
    ('baby', 0.027218255856918123),
    ('right', 0.023030601299869505),
    ('control', 0.021793028175230687),
    ('formula', 0.018982220355027904),
    ('choice', 0.018871577835331787)]
[   ('mask', 0.11875333085699452),
    ('outside', 0.06499926792588206),
    ('baby', 0.04913277398574407),
    ('wear', 0.04787096644486408),
    ('covid', 0.04755681441799534),
    ('wearing', 0.04455386834470744),
    ('merck', 0.03475969717187366),

## Topic Visualization

In [23]:
topic_model.visualize_topics()

In [24]:
topic_model.visualize_barchart(top_n_topics=14)

In [25]:
topic_model.visualize_heatmap(top_n_topics=14)

In [26]:
topic_model.visualize_hierarchy(top_n_topics=14)

In [27]:
topic_model.visualize_term_rank()

### save and load

In [197]:
topic_model.save("/content/drive/MyDrive/Colab Notebooks/PROJECTS/TOPIC-COVID/neutral_model")

/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [10]:
#topic_model = BERTopic.load("/content/drive/MyDrive/Colab Notebooks/PROJECTS/TOPIC-COVID/neutral_model")

# negative topic analysis

In [8]:
# get the tweet text and sentiment columns whre sentiment is positive as list
negative_tweet_data = data[data["sentiment"] == "negative"]
negative_tweet_data.head()


,Tweet_Id,Tweet,sentiment_analysis,sentiment,score
0,1.598104e+18,@elonmusk I thunk you're an irresponsible for ...,"('negative', 0.8987942934036255)",negative,0.898794
1,1.598104e+18,"Apparently ""I've had COVID for over a week and...","('negative', 0.5844972133636475)",negative,0.584497
2,1.598103e+18,@snakewasright @EllenOl00313689 @iTruthSearch ...,"('negative', 0.7049756646156311)",negative,0.704976
4,1.598101e+18,@1goodtern They drop dead one by one of post C...,"('negative', 0.47382649779319763)",negative,0.473826
5,1.598101e+18,@DesertCamel77 @Kate_DowlingNZ But they are ta...,"('negative', 0.811675488948822)",negative,0.811675


In [9]:
negative_tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240654 entries, 0 to 446209
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Tweet_Id            240654 non-null  float64
 1   Tweet               240654 non-null  object 
 2   sentiment_analysis  240654 non-null  object 
 3   sentiment           240654 non-null  object 
 4   score               240654 non-null  float64
dtypes: float64(2), object(3)
memory usage: 11.0+ MB


## Some Tweet Cleaning

In [10]:
negative_tweet_data["cleaned_tweet"] = negative_tweet_data["Tweet"].apply(lambda s: tweet_cleaner(s))

In [11]:
negative_tweet_data.head()

,Tweet_Id,Tweet,sentiment_analysis,sentiment,score,cleaned_tweet
0,1.598104e+18,@elonmusk I thunk you're an irresponsible for ...,"('negative', 0.8987942934036255)",negative,0.898794,thunk irresponsible allowing antivaxxers Cov...
1,1.598104e+18,"Apparently ""I've had COVID for over a week and...","('negative', 0.5844972133636475)",negative,0.584497,Apparently COVID week immunocompromised ...
2,1.598103e+18,@snakewasright @EllenOl00313689 @iTruthSearch ...,"('negative', 0.7049756646156311)",negative,0.704976,Yeah ’ easier believe Democrats eat pizza bab...
4,1.598101e+18,@1goodtern They drop dead one by one of post C...,"('negative', 0.47382649779319763)",negative,0.473826,They drop dead one one post COVID cardiac issu...
5,1.598101e+18,@DesertCamel77 @Kate_DowlingNZ But they are ta...,"('negative', 0.811675488948822)",negative,0.811675,But taking baby unmasked crowded area covid i...


In [12]:
negative_tweet_data = negative_tweet_data["cleaned_tweet"].tolist()

In [13]:
len(negative_tweet_data)

240654

## topic modelling

In [14]:
topic_model = BERTopic(verbose=True, embedding_model="all-mpnet-base-v2",  calculate_probabilities=False)

In [15]:
topics, probs = topic_model.fit_transform(negative_tweet_data)

Batches:   0%|          | 0/7521 [00:00<?, ?it/s]

2022-12-26 22:35:25,393 - BERTopic - Transformed documents to Embeddings
2022-12-26 22:52:48,075 - BERTopic - Reduced dimensionality
2022-12-26 22:53:31,631 - BERTopic - Clustered reduced embeddings


In [16]:
# after local-load start here
freq = topic_model.get_topic_info()

In [17]:
freq.head(20)

,Topic,Count,Name
0,-1,141028,-1_sick_people_get_like
1,0,4748,0_mask_wearing_wear_condom
2,1,4593,1_pro_prolife_abortion_choice
3,2,1909,2_certificate_date_passport_card
4,3,1475,3_clot_blood_pill_clotting
5,4,1387,4_boris_brexit_johnson_tory
6,5,1356,5_polio_inject_vaccinate_vaccine
7,6,1160,6_teacher_school_student_class
8,7,843,7_smell_taste_craving_sense
9,8,751,8_teenage_marriage_violence_adolescent


In [20]:
topic_model.reduce_topics(negative_tweet_data, nr_topics=14)

2022-12-26 22:56:18,125 - BERTopic - Reduced number of topics from 2030 to 15


In [21]:
import pprint
for i in range(3):
  pprint.pprint(topic_model.get_topic(i), indent=4)  # Select the most frequent 3 topic group

[   ('abortion', 0.06600480815743458),
    ('pro', 0.043833357004624984),
    ('life', 0.043229541654277844),
    ('covid', 0.03686203763461904),
    ('birth', 0.03421065617262737),
    ('choice', 0.03230898345258592),
    ('body', 0.029277840387314468),
    ('woman', 0.028710424809079295),
    ('prolife', 0.0283259159777124),
    ('people', 0.027136742102153354)]
[   ('mask', 0.1602767616056378),
    ('wear', 0.07439611585705067),
    ('wearing', 0.05601264207918296),
    ('covid', 0.04109872466780806),
    ('baby', 0.032560624615983934),
    ('people', 0.027183272232727422),
    ('pregnant', 0.024709286417334445),
    ('condom', 0.023836485410812278),
    ('get', 0.021440867700471152),
    ('like', 0.02106796334749734)]
[   ('risk', 0.07527868656448791),
    ('covid', 0.061338280736986515),
    ('woman', 0.06023533373722311),
    ('pregnant', 0.0594293145387368),
    ('death', 0.045062325781426354),
    ('pregnancy', 0.04481280257788651),
    ('higher', 0.04063375952781294),
    ('in

## Topic Visualization

In [22]:
topic_model.visualize_topics()

In [23]:
topic_model.visualize_barchart(top_n_topics=14)

In [24]:
topic_model.visualize_heatmap(top_n_topics=14)

In [25]:
topic_model.visualize_hierarchy(top_n_topics=14)

In [26]:
topic_model.visualize_term_rank()

## Save/Load BERTopic model

In [18]:
topic_model.save("/content/drive/MyDrive/Colab Notebooks/PROJECTS/TOPIC-COVID/negative_model")

/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
#topic_model = BERTopic.load("/content/drive/MyDrive/Colab Notebooks/PROJECTS/TOPIC-COVID/negative_model")

`russell-ai`

